In [34]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1, streaming=True,callbacks=[],)

poet_template = ChatPromptTemplate.from_messages([
    ("system","You are a world-famous poet,You can write a poem with any words"),
    ("human", "Write a poem about {programming_language}"),
])

critic_template = ChatPromptTemplate.from_messages([
  ("system","You are an emotional critic.So you are good at explaining poetry."),
  ("human","{poem}"),
])

poet_chain = poet_template | chat
interpreter_chain = critic_template | chat

final_chain = {"poem":poet_chain} | critic_template

final_chain.invoke({
    "programming_language": "python"
})

ChatPromptValue(messages=[SystemMessage(content='You are an emotional critic.So you are good at explaining poetry.'), HumanMessage(content='content="In the depths of the digital domain,\\nWhere code and logic intertwine,\\nThere slithers a language, elegant and refined,\\nA serpent of syntax, known as Python.\\n\\nIts scales are smooth, its movements precise,\\nGuiding programmers with effortless grace,\\nFrom data manipulation to web design,\\nPython weaves its magic in every place.\\n\\nWith libraries abundant and functions galore,\\nIt conquers tasks both big and small,\\nA versatile tool in the coder\'s hand,\\nPython answers the programmer\'s call.\\n\\nSo raise a toast to this noble snake,\\nWhose power and beauty never wane,\\nIn the world of technology, it reigns supreme,\\nPython, the poet of the digital domain."')])